<a href="https://colab.research.google.com/github/naga-Prathyusha/qml-lab-tasks/blob/main/qml_usecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# ============================================================
# Fixed — Quantum-Enhanced Drug Discovery Demo (single Colab cell)
# Paste the whole block into ONE Colab cell and run.
# ============================================================

# Install required packages (uncomment if running in fresh Colab)
# !pip install pennylane torch scikit-learn -q

import pennylane as qml
import torch
from torch import nn, optim
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import numpy as np

# ---------------------------
# 1) Synthetic "Drug Discovery" dataset (toy)
# ---------------------------
X, y = make_regression(n_samples=200, n_features=4, noise=0.1, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = (y - y.min()) / (y.max() - y.min())   # normalize to 0..1
y = y.reshape(-1, 1)

# Train/test split (simple)
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

# ---------------------------
# 2) Quantum circuit + device
# ---------------------------
n_qubits = 4
n_q_layers = 1   # number of variational layers (keep small for speed)

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_layer(inputs, weights):
    # inputs: shape (n_qubits,) (single-sample call) OR Batched by TorchLayer
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    # return expectation values for each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# ---------------------------
# 3) Hybrid model (fix: pass full weights with shape (n_layers, n_qubits, 3))
# ---------------------------
class HybridQuantumNet(nn.Module):
    def __init__(self, n_qubits, n_q_layers):
        super().__init__()
        # q_weights shape must be (n_layers, n_qubits, 3)
        # initialize small random angles
        self.q_weights = nn.Parameter(0.01 * torch.randn(n_q_layers, n_qubits, 3))
        # classical postprocessing
        self.fc1 = nn.Linear(n_qubits, 16)
        self.fc2 = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        x: tensor shape [batch, n_features] where n_features == n_qubits for encoding,
           but we'll map 4 classical features -> n_qubits using a small linear layer if needed.
        """
        batch_size = x.shape[0]

        # If input feature size != n_qubits, map it to n_qubits for encoding
        if x.shape[1] != n_qubits:
            # small linear mapping (learnable) created on the fly for simplicity
            # (for better design, define as module attribute; kept simple for demo)
            mapper = nn.Linear(x.shape[1], n_qubits).to(x.device)
            with torch.no_grad():
                # small random init so mapping isn't zero
                mapper.weight.mul_(0.1)
            x_enc = mapper(x)
        else:
            x_enc = x

        # map values to angles in [0, pi] using torch ops
        x_angles = (torch.tanh(x_enc) + 1.0) * (torch.pi / 2.0)  # shape [batch, n_qubits]

        # Run quantum circuit per sample (looping is fine for small batch/demo)
        q_outs = []
        for i in range(batch_size):
            # pass the full weights (shape (n_layers, n_qubits, 3))
            q_meas = quantum_layer(x_angles[i], self.q_weights)
            # q_meas is list-like of length n_qubits -> convert to torch tensor
            q_outs.append(torch.tensor(q_meas, dtype=torch.float32))

        q_outs = torch.stack(q_outs).to(x.device)  # shape [batch, n_qubits]

        # classical head
        h = self.relu(self.fc1(q_outs))
        out = torch.sigmoid(self.fc2(h))  # final output in [0,1]
        return out

# instantiate model
model = HybridQuantumNet(n_qubits=n_qubits, n_q_layers=n_q_layers)

# ---------------------------
# 4) Training setup
# ---------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 30

print("Training Quantum-Enhanced Drug Discovery Model...\n")
model.train()
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    preds = model(X_train_t)
    loss = criterion(preds, y_train_t)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} | Loss: {loss.item():.6f}")

# ---------------------------
# 5) Evaluation
# ---------------------------
model.eval()
with torch.no_grad():
    preds_test = model(X_test_t)
    test_loss = criterion(preds_test, y_test_t).item()

print("\nEvaluation on test split:")
print(f"Test MSE: {test_loss:.6f}\n")
print("Some sample predictions (true | pred):")
for i in range(min(8, X_test_t.shape[0])):
    print(f"{float(y_test_t[i].item()):.3f}  |  {float(preds_test[i].item()):.3f}")

print("\nDone — model ran successfully.")


Training Quantum-Enhanced Drug Discovery Model...



/tmp/ipython-input-4215491064.py:93: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  q_outs.append(torch.tensor(q_meas, dtype=torch.float32))


Epoch 01 | Loss: 0.041777
Epoch 05 | Loss: 0.030439
Epoch 10 | Loss: 0.024853
Epoch 15 | Loss: 0.024900
Epoch 20 | Loss: 0.024514
Epoch 25 | Loss: 0.023109
Epoch 30 | Loss: 0.022288

Evaluation on test split:
Test MSE: 0.035624

Some sample predictions (true | pred):
0.244  |  0.235
0.489  |  0.368
0.564  |  0.328
0.623  |  0.364
1.000  |  0.332
0.457  |  0.391
0.402  |  0.405
0.573  |  0.372

Done — model ran successfully.
